In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, BatchNormalization,Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

In [12]:
TRAINING_SIZE = 50
IMG_WIDTH, IMG_HEIGHT = 16, 16
NUM_CHANNELS = 3

In [10]:
from tf_aerial_images import extract_data, extract_labels
data_dir = 'Datasets/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, TRAINING_SIZE)
train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(train_labels) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(train_data.shape)
train_data = train_data[new_indices, :, :, :]
train_labels = train_labels[new_indices]

train_size = train_labels.shape[0]

c0 = 0
c1 = 0
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Loading Datasets/training/images/satImage_001.png
Loading Datasets/training/images/satImage_002.png
Loading Datasets/training/images/satImage_003.png
Loading Datasets/training/images/satImage_004.png
Loading Datasets/training/images/satImage_005.png
Loading Datasets/training/images/satImage_006.png
Loading Datasets/training/images/satImage_007.png
Loading Datasets/training/images/satImage_008.png
Loading Datasets/training/images/satImage_009.png
Loading Datasets/training/images/satImage_010.png
Loading Datasets/training/images/satImage_011.png
Loading Datasets/training/images/satImage_012.png
Loading Datasets/training/images/satImage_013.png
Loading Datasets/training/images/satImage_014.png
Loading Datasets/training/images/satImage_015.png
Loading Datasets/training/images/satImage_016.png
Loading Datasets/training/images/satImage_017.png
Loading Datasets/training/images/satImage_018.png
Loading Datasets/training/images/satImage_019.png
Loading Datasets/training/images/satImage_020.png


In [27]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS))
# Conv 1 3x3 16 units
c1 = Conv2D(32, (5, 5), activation='relu', kernel_initializer='truncated_normal', padding='same', use_bias=True) (inputs)
p1 = MaxPooling2D((2, 2), strides=2) (c1)

c2 = Conv2D(64, (5, 5), activation = 'relu', kernel_initializer='truncated_normal', padding='same', use_bias=True) (p1)
p2 = MaxPooling2D((2, 2), strides=2) (c2)

f1 = Flatten()(p2)
d1 = Dense(512, activation='relu', kernel_initializer='truncated_normal', use_bias=True)(f1)
d1 = Dropout(0.5)(d1)
outputs = Dense(2, activation='softmax', kernel_initializer='truncated_normal', use_bias=True)(d1)

In [34]:
model = Model(inputs=[inputs], outputs=[outputs])
adam = optimizers.Adamax(lr=1e-3)
model.compile(optimizer=adam, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 16, 16, 3)]       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 32)        2432      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               5248

In [ ]:
model.fit(train_data, train_labels, batch_size=16, validation_split=0.2, epochs=100)

Train on 12585 samples, validate on 3147 samples
Epoch 1/100
12585/12585 [==============================] - 14s 1ms/sample - loss: 0.4173 - acc: 0.8064 - val_loss: 0.7120 - val_acc: 0.7274
Epoch 2/100
12585/12585 [==============================] - 16s 1ms/sample - loss: 0.4080 - acc: 0.8086 - val_loss: 0.5385 - val_acc: 0.8179
Epoch 3/100
12585/12585 [==============================] - 14s 1ms/sample - loss: 0.4007 - acc: 0.8157 - val_loss: 0.7697 - val_acc: 0.6880
Epoch 4/100
12585/12585 [==============================] - 16s 1ms/sample - loss: 0.3925 - acc: 0.8178 - val_loss: 0.8591 - val_acc: 0.6336
Epoch 5/100
12585/12585 [==============================] - 15s 1ms/sample - loss: 0.3821 - acc: 0.8268 - val_loss: 0.9695 - val_acc: 0.5891
Epoch 6/100
12585/12585 [==============================] - 16s 1ms/sample - loss: 0.3810 - acc: 0.8253 - val_loss: 0.6793 - val_acc: 0.7461
Epoch 7/100
12585/12585 [==============================] - 15s 1ms/sample - loss: 0.3701 - acc: 0.8317 - val_lo